# Abordagens Estatísticas na Sumarização de Textos

A sumarização estatística utiliza métodos estatísticos para analisar a importância das palavras e sentenças em um texto, baseando-se em sua frequência e distribuição. Essas técnicas são especialmente úteis para identificar partes significativas do texto sem a necessidade de compreender sua estrutura linguística profunda.

Neste notebook, exploramos três abordagens estatísticas principais para a sumarização de textos:

### 1. Baseada em Frequência de Palavras

**Objetivo**: Identificar e extrair as partes mais relevantes do texto com base na frequência de ocorrência das palavras.

**Componentes**:
- **Módulo de Análise de Texto**: Decompõe o texto em palavras e calcula a frequência de cada termo.
- **Banco de Frequências**: Armazena as contagens de frequência de todas as palavras do documento.
- **Mecanismo de Seleção**: Seleciona sentenças que contêm uma alta concentração das palavras mais frequentes.

**Exemplo de Uso**: Extrair sentenças que contêm palavras que aparecem com alta frequência, indicando sua centralidade no texto.

### 2. Baseada em TF-IDF

**Objetivo**: Utilizar o método TF-IDF para diferenciar palavras que são importantes em um documento específico em relação a um corpus maior.

**Componentes**:
- **Módulo de Análise de Texto**: Analisa o texto e calcula o TF-IDF para cada palavra em cada sentença.
- **Banco de TF-IDF**: Armazena os valores de TF-IDF para todas as palavras nos documentos.
- **Mecanismo de Inferência**: Determina a relevância das sentenças com base nos valores acumulados de TF-IDF.

**Exemplo de Uso**: Sentenças que contêm palavras com alto valor de TF-IDF são consideradas importantes e selecionadas para o sumário.

### 3. Baseada em Análise de Tópicos

**Objetivo**: Identificar e resumir os principais tópicos presentes em um texto usando técnicas como LSA ou LDA para descobrir padrões semânticos.

**Componentes**:
- **Modelagem de Tópicos**: Aplica técnicas como LSA ou LDA para extrair tópicos dominantes do texto.
- **Banco de Tópicos**: Armazena informações sobre os tópicos identificados e suas associações com partes do texto.
- **Mecanismo de Seleção de Sentenças**: Escolhe sentenças que representam de forma mais eficaz os tópicos predominantes.

**Exemplo de Uso**: Selecionar sentenças que encapsulem a essência dos tópicos mais relevantes, proporcionando um sumário abrangente dos principais temas do documento.

Cada uma dessas abordagens utiliza diferentes medidas estatísticas para avaliar a importância do texto, oferecendo várias ferramentas para lidar com a complexidade e a diversidade dos documentos. A escolha da técnica depende do tipo de texto, da necessidade de profundidade na análise e da disponibilidade de recursos computacionais.


## 0. Consideração sobre NLTK

O trecho de código a seguir realiza o download dos pacotes de *stopwords* e pontuação. Verificamos na sequência as 10 primeiras *stopwords* do pacote em português.

In [1]:
import nltk

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords[:10]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

## 1. Abordagem Baseada na Frequência das Palavras

O exemplo a seguir demonstra como identificar e extrair as partes mais relevantes do texto com base na frequência de ocorrência das palavras.
Temos os seguintes elementos na implementação:
1. **Módulo de Análise de Texto**: O texto é primeiro dividido em sentenças e em palavras. As palavras são filtradas para remover stopwords e palavras não alfabéticas para focar em termos significativos.
2. **Banco de Frequências**: As frequências das palavras são calculadas e armazenadas em um Counter do módulo collections, que basicamente cria um dicionário de frequências.
3. **Mecanismo de Seleção**: Cada sentença é avaliada pela soma das frequências de suas palavras. As sentenças com os maiores scores são selecionadas para formar o sumário.

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import collections

def summarize_text_by_word_frequency(text, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)
    
    # Tokeniza o texto em palavras e remove stopwords e palavras não alfabéticas
    words = [word.lower() for word in word_tokenize(text, language=language)
             if word.isalpha() and word.lower() not in stopwords.words(language)]
    
    # Calcula a frequência de cada palavra
    word_frequencies = collections.Counter(words)
    
    # Calcula os scores de cada sentença baseando-se na soma das frequências das palavras que contém
    sentence_scores = {}
    for sentence in sentences:
        sentence_words = word_tokenize(sentence, language=language)
        score = 0
        for word in sentence_words:
            if word.lower() in word_frequencies:
                score += word_frequencies[word.lower()]
        sentence_scores[sentence] = score
    
    # Ordena as sentenças pelo score e seleciona as num_sentences mais altas
    selected_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    
    # Monta o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Exemplo de uso
example_text = """
A inteligência artificial e o aprendizado de máquina estão transformando muitas indústrias. À medida que essas tecnologias evoluem, 
seu impacto na economia continua a crescer. A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. No entanto, 
também existem preocupações sobre privacidade, segurança e o deslocamento de empregos devido à automação. Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos.
"""
summary = summarize_text_by_word_frequency(example_text)
print("Sumário Baseado na Frequência de Palavras:\n", summary)


Sumário Baseado na Frequência de Palavras:
 A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos. No entanto, 
também existem preocupações sobre privacidade, segurança e o deslocamento de empregos devido à automação.


## 2. Abordagem Baseada em TF-IDF

No exemplo de sumarização de texto utilizando o método TF-IDF, utilizamos a biblioteca scikit-learn, que fornece uma implementação eficiente de TF-IDF através do TfidfVectorizer.

1. **Módulo de Análise de Texto**: O texto é primeiramente dividido em sentenças usando sent_tokenize do NLTK, permitindo uma análise mais granular.
2. **Banco de TF-IDF**: Utilizamos TfidfVectorizer do scikit-learn para construir a matriz TF-IDF. Esta ferramenta também permite especificar stopwords em português, o que ajuda a filtrar palavras comuns e focar nos termos que realmente contribuem para a singularidade de cada sentença dentro do documento.
3. **Mecanismo de Inferência**: O score de cada sentença é calculado como a soma de seus valores TF-IDF. Sentenças com os maiores scores são consideradas as mais informativas e são escolhidas para compor o sumário.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import numpy as np

def summarize_text_by_tfidf(text, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)

    # Cria uma instância de TfidfVectorizer, especificando a língua portuguesa para stopwords
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words(language))

    # Calcula a matriz TF-IDF para as sentenças
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

    # Calcula a soma dos valores TF-IDF para cada sentença
    sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)

    # Ordena as sentenças de acordo com seus scores e seleciona as mais relevantes
    sorted_indices = np.argsort(sentence_scores)[::-1]
    selected_sentences = [sentences[idx] for idx in sorted_indices[:num_sentences]]

    # Compila o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Exemplo de uso
example_text = """
A inteligência artificial e o aprendizado de máquina estão transformando muitas indústrias. À medida que essas tecnologias evoluem, 
seu impacto na economia continua a crescer. A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. No entanto, 
também existem preocupações sobre privacidade, segurança e o deslocamento de empregos devido à automação. Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos.
"""
summary = summarize_text_by_tfidf(example_text)
print("Sumário Baseado em TF-IDF:\n", summary)


Sumário Baseado em TF-IDF:
 A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos. No entanto, 
também existem preocupações sobre privacidade, segurança e o deslocamento de empregos devido à automação.


## 3. Abordagem Baseada em LSA

Para desenvolver um exemplo de sumarização de texto que utiliza técnicas de modelagem de tópicos como Latent Semantic Analysis (LSA) ou Latent Dirichlet Allocation (LDA), podemos utilizar a biblioteca gensim para LDA ou scikit-learn para LSA. Vamos criar um exemplo usando LSA com scikit-learn, pois oferece uma integração simples com as outras ferramentas de processamento de texto.

1. **Modelagem de Tópicos**: Utiliza o TruncatedSVD, uma técnica para reduzir a dimensionalidade, aplicada à matriz TF-IDF das sentenças para extrair componentes latentes que representam tópicos no texto.
2. **Banco de Tópicos**: O fit_transform do pipeline de LSA produz uma representação dos tópicos que é armazenada em dtm_lsa.
3. **Mecanismo de Seleção de Sentenças**: As sentenças são escolhidas com base em sua associação com os principais componentes (tópicos) identificados, selecionando as que têm as maiores contribuições para o primeiro componente latente.

In [4]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from nltk.tokenize import sent_tokenize

def summarize_text_by_lsa(text, num_topics=1, num_sentences=3, language='portuguese'):
    # Tokeniza o texto em sentenças
    sentences = sent_tokenize(text, language=language)
    
    # Cria uma instância de TfidfVectorizer, especificando a língua portuguesa para stopwords
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words(language))
        
    # Calcula a matriz TF-IDF para as sentenças
    dtm = tfidf_vectorizer.fit_transform(sentences)
        
    # Configura o modelo LSA usando SVD truncado
    svd_model = TruncatedSVD(n_components=num_topics)
    lsa = make_pipeline(svd_model, Normalizer(copy=False))
    
    # Aplica LSA ao modelo DTM
    dtm_lsa = lsa.fit_transform(dtm)
    
    # Encontra as sentenças mais representativas para cada tópico
    sorted_indices = dtm_lsa[:, 0].argsort()[::-1]
    selected_sentences = [sentences[idx] for idx in sorted_indices[:num_sentences]]
    
    # Compila o sumário
    summary = ' '.join(selected_sentences)
    return summary

# Exemplo de uso
example_text = """
A inteligência artificial e o aprendizado de máquina estão transformando muitas indústrias. À medida que essas tecnologias evoluem, 
seu impacto na economia continua a crescer. A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. No entanto, 
também existem preocupações sobre privacidade, segurança e o deslocamento de empregos devido à automação. Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos.
"""
summary = summarize_text_by_lsa(example_text)
print("Sumário Baseado em LSA:\n", summary)


Sumário Baseado em LSA:
 Essas questões 
precisam ser abordadas para garantir que os benefícios das tecnologias de IA sejam maximizados enquanto minimiza impactos negativos. A integração da IA em vários setores, como saúde, finanças e educação, demonstrou benefícios significativos em termos de eficiência e produtividade. À medida que essas tecnologias evoluem, 
seu impacto na economia continua a crescer.


## 4. Análise Comparativa

In [5]:
example_text = (
    "As descobertas científicas no campo da biologia molecular estão acelerando o desenvolvimento de novos tratamentos médicos. "
    "Estes avanços são particularmente notáveis no tratamento de doenças genéticas. A tecnologia de edição genética, como CRISPR, "
    "revolucionou a medicina moderna. Além disso, a Intel lançou um novo processador que promete avanços significativos em termos "
    "de capacidade de computação e eficiência energética. A IBM anunciou uma nova parceria estratégica com a Microsoft para explorar "
    "aplicações de inteligência artificial em larga escala. Essas parcerias estão moldando o futuro das tecnologias de informação. "
    "Enquanto isso, em um estudo recente, pesquisadores descobriram um novo vírus que parece ter origem desconhecida, o que levanta "
    "preocupações sobre possíveis ameaças à saúde global. Outras pesquisas, no entanto, sugerem que o impacto desse vírus pode ser "
    "limitado a áreas específicas. As empresas de tecnologia continuam a impulsionar inovações, com muitas startups focadas no "
    "desenvolvimento de soluções sustentáveis para problemas ambientais."
)

print("Resumo Baseado em Frequência:")

summary_word_freq = summarize_text_by_word_frequency(example_text)
print(summary_word_freq)


print("\nResumo Baseado em TF-IDF:")

summary_tfidf = summarize_text_by_tfidf(example_text)
print(summary_tfidf)


print ("\nResumo Baseado em LSA:")

summary_lsa = summarize_text_by_lsa(example_text)
print(summary_lsa)

print("\nComparativo de tamanho dos textos")
print(f"\tTexto original:               {len(example_text)}")
print(f"\tResumo baseado em frequência: {len(summary_word_freq)}")
print(f"\tResumo baseado em TF-IDF:     {len(summary_tfidf)}")
print(f"\tResumo baseado em LSA:        {len(summary_lsa)}")

Resumo Baseado em Frequência:
Enquanto isso, em um estudo recente, pesquisadores descobriram um novo vírus que parece ter origem desconhecida, o que levanta preocupações sobre possíveis ameaças à saúde global. Além disso, a Intel lançou um novo processador que promete avanços significativos em termos de capacidade de computação e eficiência energética. As empresas de tecnologia continuam a impulsionar inovações, com muitas startups focadas no desenvolvimento de soluções sustentáveis para problemas ambientais.

Resumo Baseado em TF-IDF:
Enquanto isso, em um estudo recente, pesquisadores descobriram um novo vírus que parece ter origem desconhecida, o que levanta preocupações sobre possíveis ameaças à saúde global. Além disso, a Intel lançou um novo processador que promete avanços significativos em termos de capacidade de computação e eficiência energética. As empresas de tecnologia continuam a impulsionar inovações, com muitas startups focadas no desenvolvimento de soluções sustentáveis 